In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import shutil

In [2]:
# Yollar
images_npy_path = "../data/processed_data/images"
masks_npy_path = "../data/processed_data/masks"

aug_images_dir = "../data/augmented_images"
aug_masks_dir = "../data/augmented_masks"

In [2]:

# Klasörleri temizle ve oluştur
def reset_and_create_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

reset_and_create_dir(aug_images_dir)
reset_and_create_dir(aug_masks_dir)

In [ ]:
# Preprocessed verileri yükle
image_files = sorted([f for f in os.listdir(images_npy_path) if f.endswith('.npy')])
X, Y = [], []

In [4]:
for img_file in image_files:
    img_path = os.path.join(images_npy_path, img_file)
    mask_file = img_file.replace('.npy', '.npy')  # Maske isim formatı aynıysa
    mask_path = os.path.join(masks_npy_path, mask_file)
    
    if os.path.exists(mask_path):
        X.append(np.load(img_path))
        Y.append(np.load(mask_path))

X = np.array(X)
Y = np.array(Y)
print(f"Toplam Görsel: {X.shape}, Toplam Maske: {Y.shape}")

Toplam Görsel: (2594, 256, 256, 3), Toplam Maske: (2594, 256, 256)


In [5]:
# Data augmentation ayarları
data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Augmentation ve kaydetme
for i in range(len(X)):
    img = X[i][np.newaxis, ...]  # Batch boyutu ekle -> (1, 256, 256, 3)
    mask = Y[i][np.newaxis, ..., np.newaxis]  # Kanal ekle -> (1, 256, 256, 1)

    img_prefix = f"image_{i}"
    mask_prefix = f"mask_{i}"

    img_gen = image_datagen.flow(img, batch_size=1)
    mask_gen = mask_datagen.flow(mask, batch_size=1)

    for j in range(5):  # Her görsel için 5 augmentation
        augmented_img = next(img_gen)[0]
        augmented_mask = next(mask_gen)[0]

        # Augmented veriyi kaydetme
        np.save(os.path.join(aug_images_dir, f"{img_prefix}_aug_{j}.npy"), augmented_img)
        np.save(os.path.join(aug_masks_dir, f"{mask_prefix}_aug_{j}.npy"), augmented_mask)

print("Data augmentation tamamlandı ve augmented veriler kaydedildi.")



Data augmentation tamamlandı ve augmented veriler kaydedildi.


In [3]:
# Augmented verileri yükleme
def load_augmented_data(images_dir, masks_dir):
    images, masks = [], []

    for img_file in sorted(os.listdir(images_dir)):
        if img_file.endswith('.npy'):
            img_path = os.path.join(images_dir, img_file)
            mask_path = os.path.join(masks_dir, img_file.replace('image', 'mask'))
            
            if os.path.exists(mask_path):
                images.append(np.load(img_path))
                masks.append(np.load(mask_path))
                
    return np.array(images), np.array(masks)

X_aug, Y_aug = load_augmented_data(aug_images_dir, aug_masks_dir)
X_train, X_val, Y_train, Y_val = train_test_split(X_aug, Y_aug, test_size=0.2, random_state=42)

print(f"Eğitim Seti: {X_train.shape}, Doğrulama Seti: {X_val.shape}")

Eğitim Seti: (10376, 256, 256, 3), Doğrulama Seti: (2594, 256, 256, 3)


In [4]:
def unet_with_resnet50(input_shape=(256, 256, 3), dropout_rate=0.5, l2_reg=1e-4):
    inputs = Input(shape=input_shape)

    # ResNet50 Encoder
    resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    resnet.trainable = False

    # Encoder katmanları
    s1 = resnet.get_layer("conv1_relu").output      # 128x128
    s2 = resnet.get_layer("conv2_block3_out").output  # 64x64
    s3 = resnet.get_layer("conv3_block4_out").output  # 32x32
    s4 = resnet.get_layer("conv4_block6_out").output  # 16x16
    b = resnet.get_layer("conv5_block3_out").output   # 8x8

    # Decoder kısmı
    d1 = UpSampling2D()(b)  # 16x16
    d1 = concatenate([d1, s4])
    d1 = Dropout(dropout_rate)(Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(d1))

    d2 = UpSampling2D()(d1)  # 32x32
    d2 = concatenate([d2, s3])
    d2 = Dropout(dropout_rate)(Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(d2))

    d3 = UpSampling2D()(d2)  # 64x64
    d3 = concatenate([d3, s2])
    d3 = Dropout(dropout_rate)(Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(d3))

    d4 = UpSampling2D()(d3)  # 128x128
    d4 = concatenate([d4, s1])
    d4 = Dropout(dropout_rate)(Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(d4))

    d5 = UpSampling2D()(d4)  # 256x256
    d5 = Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=l2(l2_reg))(d5)

    # Çıkış katmanı
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(d5)

    model = Model(inputs, outputs)
    return model

# Modeli oluştur
model = unet_with_resnet50()
model.summary()

2024-12-15 19:37:27.727064: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-12-15 19:37:27.728311: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-15 19:37:27.728317: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-15 19:37:27.728363: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-15 19:37:27.728435: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 262, 262,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 128, 128,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 128, 128,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 128, 128,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 130, 130,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 64, 64,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 64, 64,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 64, 64,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 64, 64,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 64, 64,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 64, 64,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 64, 64,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 64, 64,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 64, 64,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_3_c

 Total params: 31,813,505 (121.36 MB)

 Trainable params: 8,225,793 (31.38 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('../models2/unet_augmented.keras', save_best_only=True)
]

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=16,
    batch_size=16,
    callbacks=callbacks
)

Epoch 1/16


2024-12-15 19:39:08.349785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/tensorflow-metal/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


649/649 ━━━━━━━━━━━━━━━━━━━━ 114s 176ms/step - val_accuracy: 0.7659 - val_loss: 10.6908
Epoch 2/16


KeyboardInterrupt: 